In [2]:
import csv
import numpy as np
from ultralytics import YOLO
import cv2
import os
import pandas as pd
import tqdm
import torch
from torch.utils.data import Dataset, DataLoader

In [2]:
detect_model = YOLO('./weights/yolov8x.pt')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
df = pd.read_csv('../data/frames.csv')

cols = df.columns.tolist()


cols.insert(6, 'box_x')
cols.insert(7, 'box_y')
cols.insert(8, 'box_w')
cols.insert(9, 'box_h')

In [4]:
class ImageDataset(Dataset):
    def __init__(self, df, root_path):
        self.df = df
        self.root_path = root_path

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.root_path, row['img_path'])
        img = cv2.imread(img_path)
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (640, 640))
        img = img / 255.0  # Normalize
        img_tensor = torch.from_numpy(img).permute(2, 0, 1).float()  # Convert to tensor and permute dimensions
        return img_tensor, idx

In [25]:
root_path = '../data/frames'
batch_size = 64
new_df = pd.DataFrame(columns=cols)

dataset = ImageDataset(df, root_path)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# Process batches from the DataLoader
for batch in tqdm.tqdm(dataloader):
    images, idxs = batch
    images.to(device)
    results = detect_model(images, classes=[0],verbose=False, device=device)

    idxs = idxs.numpy()
    for i, idx in enumerate(idxs):
        row = df.iloc[idx]
        
        boxes = results[i].boxes
        if len(boxes.xywh) == 0:
            continue
        xywh = boxes.xywh.cpu().numpy()[0]
        scale_w = 1150 / 640
        scale_h = 1800 / 640
        cx, cy, w, h = xywh
        xywh = [cx-w/2, cy-h/2, w, h]
        xywh = [xywh[0] * scale_w, xywh[1] * scale_h, xywh[2] * scale_w, xywh[3] * scale_h]
       
        row_dict = row.copy().to_dict()

        row_dict['box_x'] = round(xywh[0], 2)
        row_dict['box_y'] = round(xywh[1], 2)
        row_dict['box_w'] = round(xywh[2], 2)
        row_dict['box_h'] = round(xywh[3], 2)
        
        new_df = pd.concat([new_df, pd.DataFrame([row_dict])], ignore_index=True)

new_df.to_csv('../data/detect.csv', index=False)

  0%|          | 0/1125 [00:00<?, ?it/s]/tmp/ipykernel_4789/897782342.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, pd.DataFrame([row_dict])], ignore_index=True)
100%|██████████| 1125/1125 [2:20:54<00:00,  7.52s/it] 
